# eShopOnWeb sample

Das Projekt eShopOnWeb ist eine Beispiel-Webanwendung, die als Referenz für die Entwicklung von E-Commerce-Anwendungen dient. Es nutzt .NET Core und demonstriert eine einfache Architektur mit verschiedenen modernen Webtechnologien. Die Anwendung enthält typische Funktionalitäten wie Produktauswahl, Einkaufswagen und Bestellmanagement und eignet sich hervorragend, um mehr über den Aufbau von Webanwendungen und die Nutzung von DevOps-Technologien wie Docker und Devcontainers zu lernen.

In diesem Notebook werden wir Schritt für Schritt erklären, wie man das eShopOnWeb-Projekt:
* 1. Klonen
* 2. Lokal ausführen
* 3. Im Devcontainer starten
* 4. In Docker ausführen kann.
* 5. Auf Azure ausrollen kann.

Hier ist eine ausführliche Beschreibung der Schritte, wie du das eShopOnWeb-Projekt lokal, im Devcontainer, in Docker und auf Azure ausrollen kannst:

### 1. **Repository Klonen**

Zunächst musst du das eShopOnWeb-Repository von GitHub auf deinen lokalen Rechner klonen. Dies geht einfach über den Git-Befehl:

In [1]:
!git clone https://github.com/NimblePros/eShopOnWeb.git

Cloning into 'eShopOnWeb'...
remote: Enumerating objects: 9456, done.
remote: Counting objects: 100% (1104/1104), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 9456 (delta 603), reused 1020 (delta 555), pack-reused 8352 (from 1)
Receiving objects: 100% (9456/9456), 62.29 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (5993/5993), done.


Dieser Schritt holt das Projekt in deinen lokalen Ordner.

### 2. **Lokales Starten des Projekts**

Um das Projekt lokal auszuführen, musst du sicherstellen, dass du das .NET SDK installiert hast. Falls du es noch nicht hast, kannst du es von der offiziellen .NET-Website herunterladen:  
[Download .NET SDK](https://dotnet.microsoft.com/download/dotnet).

#### 2.1. (optional) Änderungen für VSCode auf MacOS

<details><summary>Zum Nachlesen bitte aufklappen...</summary>


Um den beschriebenen DIFF in deinem Projekt manuell umzusetzen, solltest du folgende Schritte durchführen:

### 1. **Änderungen an der `docker-compose.yml`**:
   - **Füge das `^M`-Zeichen nach bestimmten Zeilen hinzu** (dies könnte ein Zeilenumbruchproblem sein, das du eventuell in einem bestimmten Editor siehst). Beachte, dass dieses Zeichen auf Windows-basierten Systemen auftreten kann.
   - Ändere die `SA_PASSWORD` Umgebungsvariable:
     - Ändere `SA_PASSWORD=@someThingComplicated1234` zu `MSSQL_SA_PASSWORD=Some$Complex1234Password`.
   
   Beispieländerung:
   ```yaml
   services:
     sqlserver:
       image: mcr.microsoft.com/azure-sql-edge
       ports:
         - "1433:1433"
       environment:
         - MSSQL_SA_PASSWORD=Some$Complex1234Password
         - ACCEPT_EULA=Y
   ```

### 2. **Änderungen an der `src/PublicApi/Dockerfile`**:
   - Ändere die Basis- und Build-Image-Version von .NET 8.0 auf .NET 9.0. Ändere die Zeilen:
     - `FROM mcr.microsoft.com/dotnet/aspnet:8.0 AS base` zu `FROM mcr.microsoft.com/dotnet/aspnet:9.0 AS base`
     - `FROM mcr.microsoft.com/dotnet/sdk:8.0 AS build` zu `FROM mcr.microsoft.com/dotnet/sdk:9.0 AS build`
   
   Beispieländerung:
   ```Dockerfile
   FROM mcr.microsoft.com/dotnet/aspnet:9.0 AS base
   FROM mcr.microsoft.com/dotnet/sdk:9.0 AS build
   ```

### 3. **Änderungen an der `src/Web/Dockerfile`**:
   - Ändere ebenfalls die .NET-Versionen in der `Dockerfile` von Version 8.0 auf 9.0:
     - `FROM mcr.microsoft.com/dotnet/sdk:8.0 AS build` zu `FROM mcr.microsoft.com/dotnet/sdk:9.0 AS build`
     - `FROM mcr.microsoft.com/dotnet/aspnet:8.0 AS runtime` zu `FROM mcr.microsoft.com/dotnet/aspnet:9.0 AS runtime`
   
   Beispieländerung:
   ```Dockerfile
   FROM mcr.microsoft.com/dotnet/sdk:9.0 AS build
   FROM mcr.microsoft.com/dotnet/aspnet:9.0 AS runtime
   ```

### 4. **Änderungen an der `src/Web/appsettings.json`**:
   - Füge einen neuen Abschnitt für die `UseOnlyInMemoryDatabase` Konfiguration hinzu:
   
   Beispieländerung:
   ```json
   {
     "ConnectionStrings": {
       "DefaultConnection": "..."
     },
     "Logging": {
       "LogLevel": {
         "Default": "Information",
         "System": "Warning"
       }
     },
     "AllowedHosts": "*",
     "UseOnlyInMemoryDatabase": true
   }
   ```

### Zusammenfassung:
- Aktualisiere die `.NET`-Version in den Dockerfiles auf `9.0` statt `8.0`.
- Ändere die Umgebungsvariable für das SQL-Server-Passwort in der `docker-compose.yml`.
- Füge eine neue Konfiguration `UseOnlyInMemoryDatabase` in der `appsettings.json` hinzu.

Das sind die manuellen Änderungen, die du vornehmen musst, um den gewünschten DIFF zu erreichen.
</details>

#### 2.2. Abhängigkeiten wiederherstellen

Nach dem Klonen des Projekts musst du zunächst alle Abhängigkeiten wiederherstellen:

In [38]:
!dotnet restore eShopOnWeb/eShopOnWeb.sln > /dev/null

=

Dieser Befehl lädt alle notwendigen NuGet-Pakete herunter, die das Projekt benötigt.

#### 2.3. Projekt bauen

Nun kannst du das Projekt bauen, um sicherzustellen, dass keine Fehler vorhanden sind:

In [39]:
!dotnet build eShopOnWeb/eShopOnWeb.sln > /dev/null

==

#### 2.4. Projekt ausführen

Auf einem Windows basierten System kannst du das Projekt nun lokal starten:

##### 1. Starte das Backend:

Öffne einen Terminal und führe folgende Befehle aus:

```bash
cd notebooks/samples
dotnet run --project eShopOnWeb/src/PublicApi
```

Die API sollte nun unter `http://localhost:5099/api` im Browser erreichbar sein.

##### 2. Starte nun das Frontend:

Öffne einen weiteren Terminal und führe folgende Befehle aus:

```bash
cd notebooks/samples
dotnet run --project eShopOnWeb/src/Web
```

Das Projekt sollte nun unter `http://localhost:5001` im Browser verfügbar sein.

### 3. **Devcontainer Starten**

Ein Devcontainer ist eine Docker-basierte Entwicklungsumgebung, die in Visual Studio Code (VSCode) verwendet werden kann. Um das Projekt in einem Devcontainer zu starten, folge diesen Schritten:

#### 3.1. VSCode Devcontainer Erweiterung installieren

Stelle sicher, dass du die **Dev Containers**-Erweiterung in VSCode installiert hast. Dies ermöglicht es dir, das Projekt direkt im Container zu entwickeln.

#### 3.2. Projekt im Devcontainer öffnen

- Öffne das geklonte `eShopOnWeb`-Projekt in VSCode.
- Wähle in VSCode die Option **Reopen in Container**, um das Projekt im Devcontainer zu starten. 

VSCode baut dann den Container und stellt sicher, dass die Entwicklungsumgebung genau der entspricht, die im Projekt konfiguriert ist. Der Devcontainer enthält alle benötigten Tools und Abhängigkeiten.

### 4. **Docker Starten**

Falls du das Projekt in einem Docker-Container ausführen möchtest, kannst du dies tun, indem du ein Docker-Image erstellst und es im Container ausführst.

#### 4.1. Docker installieren

Wenn Docker noch nicht auf deinem Computer installiert ist, kannst du es von der offiziellen Website herunterladen:  
[Download Docker](https://www.docker.com/get-started).

#### 4.2. Docker-Image bauen

Im Projektordner kannst du mit folgendem Befehl ein Docker-Image erstellen:

In [40]:
!docker compose -f ./eShopOnWeb/docker-compose.yml build > /dev/null

WARN[0000] The "Complex1234Password" variable is not set. Defaulting to a blank string. 
WARN[0000] /Users/starkj/Documents/2hands/Soproming/Repo/main/notebooks/samples/eShopOnWeb/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 


#### 4.3. Docker-Container starten

Sobald das Image gebaut ist, kannst du den Container starten:

In [ ]:
!docker compose -f ./eShopOnWeb/docker-compose.yml up > /dev/null

WARN[0000] The "Complex1234Password" variable is not set. Defaulting to a blank string. 
WARN[0000] /Users/starkj/Documents/2hands/Soproming/Repo/main/notebooks/samples/eShopOnWeb/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 0/0
 ⠙ sqlserver Pulling                                                       0.1s 
[+] Running 0/1
 ⠹ sqlserver Pulling                                                       0.2s 
[+] Running 0/1
 ⠸ sqlserver Pulling                                                       0.3s 
[+] Running 0/1
 ⠼ sqlserver Pulling                                                       0.4s 
[+] Running 0/1
 ⠴ sqlserver Pulling                                                       0.5s 
[+] Running 0/1
 ⠦ sqlserver Pulling                                                       0.6s 
[+] Running 0/1
 ⠧ sqlserver Pulling                                                       0.7s 
[+] Running 0/1
 

Dies startet das Projekt im Docker-Container, und du kannst die Anwendung unter `localhost:5106` for das Frontend und unter `localhost:5200` im Browser erreichen.

### 5. **Projekt auf Azure ausrollen**

Für das Deployment auf Azure benötigst du ein Azure-Konto und die **Azure CLI**. Falls du sie noch nicht installiert hast, folge diesen Anweisungen:  
[Installiere Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli).

#### 5.1. Voraussetzungen

- Ein Azure-Konto: [Azure Kostenloses Konto](https://azure.microsoft.com/free)
- Azure CLI installiert: [Azure CLI Installieren](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)
- Docker (wenn du Docker für das Deployment verwenden möchtest).
- Ein App Service Plan auf Azure (für das Hosting der Webanwendung).
- (Optional) Eine registrierte Anwendung in Azure Active Directory, falls du Authentifizierung benötigst.

#### 5.2. Projekt für das Azure-Deployment vorbereiten

Bevor du das Projekt auf Azure ausrollst, solltest du sicherstellen, dass alle Umgebungsvariablen wie Datenbank-Verbindungszeichenfolgen und geheime Werte korrekt gesetzt sind. Du solltest diese nicht im Code speichern, sondern über **App Settings** in Azure verwalten.

#### 5.3. Azure Deployment mit Azure CLI

Nun kannst du das Projekt mit Azure CLI auf Azure bereitstellen. Folge diesen Schritten (diese sind aktuell ungestestet):

##### 5.3.1. Anmelden bei Azure

Melde dich bei deinem Azure-Konto an:

```bash
az login
```

##### 5.3.2. Erstellen einer Ressourcengruppe

Erstelle eine neue Ressourcengruppe, die alle Ressourcen für das Projekt enthält:

```bash
az group create --name eShopOnWebResourceGroup --location "East US"
```

##### 5.3.3. Erstellen des App Service Plans

Der App Service Plan definiert die Infrastruktur, auf der deine Web-App laufen wird. Erstelle ihn mit:

```bash
az appservice plan create --name eShopOnWebPlan --resource-group eShopOnWebResourceGroup --sku B1 --is-linux
```

##### 5.3.4. Erstellen der Web-App

Erstelle eine neue Web-App, die deine Anwendung hosten wird. Hier kannst du die Laufzeit angeben (z. B. .NET 6):

```bash
az webapp create --resource-group eShopOnWebResourceGroup --plan eShopOnWebPlan --name eShopOnWebApp --runtime "DOTNET|6.0"
```

##### 5.3.5. Docker-Deployment (Optional)

Wenn du Docker verwendest, kannst du das Docker-Image auf Azure Web App for Containers hochladen:

```bash
az webapp config container set --name eShopOnWebApp --resource-group eShopOnWebResourceGroup --docker-custom-image-name eshoponweb
```

##### 5.3.6. Git-Deployment (Für lokale .NET-Anwendung)

Falls du kein Docker verwendest, kannst du das Projekt auch direkt mit Git auf Azure deployen:

```bash
az webapp deployment source config-local-git --name eShopOnWebApp --resource-group eShopOnWebResourceGroup
```

#### 5.4. Überprüfung der Bereitstellung

Nach erfolgreichem Deployment kannst du die Anwendung über die folgende URL aufrufen:

```text
https://<dein-app-name>.azurewebsites.net
```

### 6. **Fehlerbehebung und mögliche Probleme**

- **Bauen des Projekts**: Wenn beim Bauen des Projekts Fehler auftreten, überprüfe, ob alle Abhängigkeiten korrekt installiert sind und ob du das richtige .NET SDK verwendest.
- **Docker**: Wenn Docker nicht funktioniert, überprüfe, ob Docker korrekt installiert und der Docker-Daemon läuft.
- **Azure Deployment**: Wenn das Azure Deployment fehlschlägt, stelle sicher, dass du alle Ressourcen korrekt erstellt hast und die Umgebungsvariablen für die Web-App richtig konfiguriert sind. Prüfe auch, ob du die richtige Runtime und Region für deinen App Service Plan ausgewählt hast.

Mit diesen Schritten solltest du in der Lage sein, das eShopOnWeb-Projekt lokal, im Devcontainer, in Docker und auf Azure auszurollen.